In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import libraries
import sys

sys.path.append("../src/bcc_model/")

import itertools
import math
from tqdm.autonotebook import tqdm


import pandas as pd
from v2.blue_carbon_project import BlueCarbonProject
from v2.cost_calculator import CostCalculator
from v2.sequestration_credits_calculator import SequestrationCreditsCalculator
from v2.utils import generate_master_table

/tmp/ipykernel_455362/1444598790.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Define functions:
def calculate_breakeven_cost(project, max_iterations=500, tolerance=1e-5):
    """
    Function to calculate the breakeven cost of carbon
    """
    # Start the calculation with the initial carbon price
    carbon_price = project.carbon_price

    for _iteration in range(max_iterations):
        # Update the project carbon price and initialize cost calculator
        project.carbon_price = carbon_price
        cost_calculator = CostCalculator(project)

        # Calculate NPV covering cost
        npv_covering_cost = cost_calculator.NPV_covering_cost
        credits_issued = cost_calculator.credits_issued

        # Check if the NPV covering cost is within the acceptable tolerance
        if abs(npv_covering_cost) < tolerance:
            # Retrieve additional cost estimates and summary
            cost_summary = cost_calculator.get_summary(table=False)  # Assuming this method exists
            cost_estimates = cost_calculator.get_cost_estimates(
                table=False
            )  # Assuming this method exists

            return {
                "breakeven_carbon_price": float(carbon_price),
                "cost_summary": cost_summary,
                "cost_estimates": cost_estimates,
            }

        # Ensure credits_issued is not zero to avoid division errors
        if credits_issued == 0:
            print("Error: Credits issued are zero, breakeven cost cannot be calculated.")
            return None

        # Update carbon price based on the NPV covering cost and credits issued
        carbon_price -= npv_covering_cost / credits_issued

    # If max_iterations are reached without convergence, return the last calculated price
    print("Warning: Max iterations reached without convergence.")
    return {
        "breakeven_carbon_price": float(carbon_price),
        "cost_summary": cost_calculator.get_summary(table=False),
        "cost_estimates": cost_calculator.get_cost_estimates(table=False),
    }

In [4]:
# Paths
master_data_path = "../excel/Carbon-Cost Data Upload.xlsm"
output_path = "../raw_data/output_v2.csv"
raw_scorecard_data_path = "../raw_data/scorecard_rating/qualitative_scorecard_rating.csv"
output_scorecard = "../raw_data/output_scoring.csv"  # this needs to generate the ../excel/data_ingestion_project_scorecard.xlsm


In [5]:
# 1. Import master table, base size table and base increase table
#  with information for the project calculations
# data_path = "../raw_data/data_ingestion.xlsm"
# Import excel with the import datai
master_table = generate_master_table(master_data_path)
# Open the excel file - tab 'Base_size_table'
base_size = pd.read_excel(master_data_path, sheet_name="Base_size_table")

# Open the excel file - tab 'Base_increase'
base_increase = pd.read_excel(master_data_path, sheet_name="Base_increase")

# change in the country column "Caribbean" to "The Bahamas"
master_table["country"] = master_table["country"].replace("Caribbean", "The Bahamas")

master_table.head()

,country,country_code,activity,ecosystem,size_ha,feasibility_analysis_cost,planning_and_admin_cost,data_collection_and_field_cost,community_representation_liaison_cost,blue_carbon_project_planning_cost,...,extent,extent_historic,unprotected_extent,loss_rate,restorable_land,tier_1_ipcc_default_value,tier_2_country_specific_rate,tier_1_global_emission_factor,tier_2_country_specific_emission_factor_agb,tier_2_country_specific_emission_factor_soc
0,United States,USA,Restoration,Mangrove,500,100000,166766.666667,26666.666667,126500.000000,100000,...,2.315054e+05,2.312725e+05,20077.0,-0.0089,9666.0,34.933,17.2349,27.2,71.6,58.0
1,Indonesia,IDN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,71183.333333,100000,...,2.901690e+06,2.930508e+06,2028221.0,-0.0037,204536.0,34.933,64.5100,27.2,460.5,71.9
2,Australia,AUS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,113016.666667,115000,...,1.004149e+06,1.013552e+06,482832.0,-0.0015,69160.0,34.933,7.2850,27.2,256.4,59.8
3,The Bahamas,BHS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,103100.000000,125000,...,1.508391e+05,1.447886e+05,34036.0,-0.0073,9842.0,34.933,10.5160,27.2,28.3,107.9
4,Kenya,KEN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,66050.000000,100000,...,5.412150e+04,5.408001e+04,44742.0,-0.0028,1610.0,34.933,24.9400,27.2,118.9,79.0


In [6]:
# 2. Define different list with all the possible values for the different parameters

# Definition of lists
country_list = [
    "United States",
    "Indonesia",
    "Australia",
    "The Bahamas",
    "Kenya",
    "Mexico",
    "Colombia",
    "India",
    "China",
]
# Include country size information to normalize the total cost
# Idially we will want to have this information in the country entity
country_size_ha = {
    "United States": 947084624.2706754,
    "Indonesia": 188785480.2259437,
    "Australia": 768882542.08165,
    "The Bahamas": 1338557.8163060332,
    "Kenya": 58606174.82755706,
    "Mexico": 195179334.58619106,
    "Colombia": 113742621.27637246,
    "India": 297769359.954299,
    "China": 934894938.3876103,
}
ecosystem_list = ["Mangrove", "Seagrass", "Salt marsh"]
activity_type_list = ["Restoration", "Conservation"]
restoration_activity_type_list = ["Planting", "Hydrology", "Hybrid"]

# Definition of parameters
project_size_filter = ["Small", "Medium", "Large"]
price_type = ["Opex breakeven", "Market price"]

# Define project size thresholds for each ecosystem and activity type
project_size_thresholds = {
    ("Mangrove", "Conservation"): {"Small": 4000, "Medium": 20000, "Large": 40000},
    ("Mangrove", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Salt marsh", "Conservation"): {"Small": 800, "Medium": 4000, "Large": 8000},
    ("Salt marsh", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Seagrass", "Conservation"): {"Small": 400, "Medium": 2000, "Large": 4000},
    ("Seagrass", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
}

In [7]:
# 3. Create all possible combinations for the overview page
all_combinations = []

for country, ecosystem, activity, project_size, price in itertools.product(
    country_list, ecosystem_list, activity_type_list, project_size_filter, price_type
):
    # Apply restoration activity type only if the activity is "Restoration"
    if activity == "Restoration":
        for restoration_activity in restoration_activity_type_list:
            threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
            all_combinations.append(
                {
                    "country": country,
                    "ecosystem": ecosystem,
                    "activity": activity,
                    "activity_type": restoration_activity,
                    "project_size_filter": project_size,
                    "project_size": threshold,
                    "price_type": price,
                }
            )
    else:
        # For Conservation, restoration activity type should be None
        threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
        all_combinations.append(
            {
                "country": country,
                "ecosystem": ecosystem,
                "activity": activity,
                "activity_type": None,
                "project_size_filter": project_size,
                "project_size": threshold,
                "price_type": price,
            }
        )

# Add country size information to the combinations
for combination in all_combinations:
    combination["country_size_ha"] = country_size_ha[combination["country"]]

# Convert to DataFrame
df_combinations = pd.DataFrame(all_combinations)

# Display the first few rows of the DataFrame
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08
1,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08
2,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08
3,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08
4,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08


In [8]:
# 4. Apply model constrains

# Remove all rows for Seagrass restoration projects that are not Planting
df_combinations = df_combinations[
    ~(
        (df_combinations["ecosystem"] == "Seagrass")
        & (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] != "Planting")
    )
]

# Define conditions for removal based on country, ecosystem, activity, and activity_type
conditions_to_remove = [
    # List of country-ecosystem pairs to remove 'Hybrid' restoration activities
    # Remove the projects for Indonesia Manrgove Restoaration Hybrid
    # Remove projects for Indonesia Salt Marsh Restoration Hybrid
    # Remove projects for Australia Mangrove Restoration Hybrid
    # Remove pprojects for Australia Salt Marsh Restoration Hybrid
    # Remove projects for Caribbean Mangrove Restoration Hybrid
    # Remove project for Caribbean Salt marsh Restoration Hybrid
    # Remove projects for Kenya Mangrove Restoration Hybrid
    # Remove projects for Kenya Salt marsh Restoration Hybrid
    # Remove projects for Colombia mangrove restoration hybrid
    # Remove projects for Colombia Salt marsh Restoration Hybrid
    # Remove prjects India Manrgove restoration Hybrid
    # Remove projects for India Salt marsh Restoration Hybrid
    # Remove projects for China Mangrove Restoration Hybrid
    # Remove projects for China Salt marsh Restoration Hybrid
    ("Indonesia", "Mangrove"),
    ("Indonesia", "Salt marsh"),
    ("Australia", "Mangrove"),
    ("Australia", "Salt marsh"),
    ("Caribbean", "Mangrove"),
    ("Caribbean", "Salt marsh"),
    ("Kenya", "Mangrove"),
    ("Kenya", "Salt marsh"),
    ("Colombia", "Mangrove"),
    ("Colombia", "Salt marsh"),
    ("India", "Mangrove"),
    ("India", "Salt marsh"),
    ("China", "Mangrove"),
    ("China", "Salt marsh"),
]

# 3. Apply the conditions to filter out specified rows in a single operation
df_combinations = df_combinations[
    ~(
        (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] == "Hybrid")
        & (
            df_combinations[["country", "ecosystem"]]
            .apply(tuple, axis=1)
            .isin(conditions_to_remove)
        )
    )
]

In [9]:
# 5. Get the country name, country_code and continent information from the master table:

# Filter master table just for the columns we need
master_table_filtered = master_table[["country", "country_code"]]
master_table_filtered.head()

# left merge the master_table with the df_combinations
df_combinations = df_combinations.merge(master_table_filtered, how="left", on=["country"])
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA


In [10]:
# 6. Get the project_size_ha from the master_table and rename it to base_size
base_size_value_df = master_table[["country_code", "ecosystem", "activity", "size_ha"]]

# Merge base_size_value_df with df_combinations
df_combinations = df_combinations.merge(
    base_size_value_df, how="left", on=["country_code", "ecosystem", "activity"]
)
# Rename the column project_size_ha to base_size
df_combinations.rename(
    columns={"size_ha": "base_size", "project_size": "project_size_ha"}, inplace=True
)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size_ha,price_type,country_size_ha,country_code,base_size
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500


In [11]:
# 7. Create a new column called "project_name" by concatenating the "country", "ecosystem",
# "activity" and "project_size" columns
df_combinations["project_name"] = (
    df_combinations["country"]
    + " "
    + df_combinations["ecosystem"]
    + " "
    + df_combinations["activity"]
    + " "
    + df_combinations["activity_type"].fillna("")
    + " "
    + df_combinations["project_size_filter"].astype(str)
)
# drop duplicates in case we have some duplicated project names
df_combinations.drop_duplicates(inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size_ha,price_type,country_size_ha,country_code,base_size,project_name
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration Planting Small
6,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration Hydrology S...
12,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration Hybrid Small
18,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08,USA,500,United States Mangrove Restoration Planting Small
24,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08,USA,500,United States Mangrove Restoration Hydrology S...


In [12]:
#
columns_to_add = [
    "total_cost_npv",
    "total_cost",
    "total_weighted_cost_npv",
    "total_weighted_cost",
    "capex_npv",
    "capex",
    "opex_npv",
    "opex",
    "country_abatement_potential",
    "project_abatement_potential",
    "cost_per_tco2e",
    "cost_per_tco2e_npv",
    "feasibility_analysis_npv",
    "feasibility_analysis",
    "conservation_planning_npv",
    "conservation_planning",
    "data_collection_npv",
    "data_collection",
    "community_representation_npv",
    "community_representation",
    "blue_carbon_project_planning_npv",
    "blue_carbon_project_planning",
    "establishing_carbon_rights_npv",
    "establishing_carbon_rights",
    "validation_npv",
    "validation",
    "implementation_labor_npv",
    "implementation_labor",
    "monitoring_maintenance_npv",
    "monitoring_maintenance",
    "community_benefit_npv",
    "community_benefit",
    "carbon_standard_fees_npv",
    "carbon_standard_fees",
    "baseline_reassessment_npv",
    "baseline_reassessment",
    "mrv_npv",
    "mrv",
    "long_term_project_operating_npv",
    "long_term_project_operating",
    "initial_price_assumption",
    "leftover_after_opex",
    "leftover_after_opex_npv",
    "total_revenue",
    "total_revenue_npv",
    "credits_issued",
]


In [13]:
# 8. Run the model for each project and calculate the costs
project_base_params = {
    "planting_success_rate": 0.6,  # Success rate for planting change in v2
    "loss_rate_used": "National average",
    "master_table": master_table,
    "base_size": base_size,
    "base_increase": base_increase,
    "carbon_revenues_to_cover": "Opex",
}
# Step 1: Initialize columns for the output df with None

df_combinations[columns_to_add] = None
# Step 2: Loop through each project and calculate costs
for index, row in tqdm(df_combinations.iterrows(), total=df_combinations.shape[0]):
    projet_name = row["project_name"]
    # print(f"Calculating project {projet_name}...")

    # Set up row BlueCarbonProject parameters
    project_params = project_base_params.copy()
    project_params.update(
        {
            "activity": row["activity"],
            "ecosystem": row["ecosystem"],
            "country": row["country"],
            "project_size_ha": row["project_size_ha"],
            "restoration_activity": row["activity_type"],
        }
    )

    # Set ecosystem-specific rates
    if row["ecosystem"] == "Mangrove":
        project_params.update(
            {
                "sequestration_rate_used": "Tier 2 - Country-specific rate",
                "emission_factor_used": "Tier 2 - Country-specific emission factor",
            }
        )
    else:
        # print(f"No default Tier 2 sequestration rate data available for {row['ecosystem']}.")
        project_params.update(
            {
                "sequestration_rate_used": "Tier 1 - IPCC default value",
                "emission_factor_used": "Tier 1 - Global emission factor",
            }
        )

    # Determine the initial carbon price based on price type
    initial_price_assumption = 30
    project_params["carbon_price"] = initial_price_assumption

    # Initialize project with the initial or market price
    project = BlueCarbonProject(**project_params)

    if row["price_type"] == "Market price":
        df_combinations.loc[index, "initial_price_assumption"] = initial_price_assumption

    elif row["price_type"] == "Opex breakeven":
        try:
            breakeven = calculate_breakeven_cost(project, max_iterations=150)
            breakeven_cost = breakeven["breakeven_carbon_price"]
            # print(f"Breakeven cost for project {row['project_name']} is {breakeven_cost}")
            df_combinations.loc[index, "initial_price_assumption"] = breakeven_cost
            # summary = breakeven['cost_summary']
            # cost_estimates = breakeven['cost_estimates']
            # Update project_params with breakeven carbon price
            project_params["carbon_price"] = breakeven_cost

            # Re-initialize project with breakeven price
            project = BlueCarbonProject(**project_params)
        except Exception as e:  # noqa: E722
            print(
                f"Error: Breakeven cost for project {row['project_name']} could not be calculated {e}"
            )
            df_combinations.loc[index, "initial_price_assumption"] = initial_price_assumption
            # Re-initialize project with the initial price if breakeven calculation fails
            project_params["carbon_price"] = (
                initial_price_assumption  # <-- Set to initial price if error
            )
            project = BlueCarbonProject(**project_params)
            # cost_calculator = CostCalculator(project)
            # summary = cost_calculator.get_summary()
            # cost_estimates = cost_calculator.get_cost_estimates()

    # print('project params:', project_params)
    # Step 3: Calculate costs and extract data into dictionary
    cost_calculator = CostCalculator(project)
    cost_estimates = cost_calculator.get_cost_estimates(table=False)
    summary = cost_calculator.get_summary(table=False)

    # Define a dictionary for cost components mapping
    cost_mapping = {
        "total_cost_npv": ("Total cost", "NPV"),
        "total_cost": ("Total cost", "Total cost"),
        "capex_npv": ("Capital expenditure", "NPV"),
        "capex": ("Capital expenditure", "Total cost"),
        "opex_npv": ("Operating expenditure", "NPV"),
        "opex": ("Operating expenditure", "Total cost"),
        "feasibility_analysis_npv": ("Feasibility analysis", "NPV"),
        "feasibility_analysis": ("Feasibility analysis", "Total cost"),
        "conservation_planning_npv": ("Conservation planning and admin", "NPV"),
        "conservation_planning": ("Conservation planning and admin", "Total cost"),
        "data_collection_npv": ("Data collection and field costs", "NPV"),
        "data_collection": ("Data collection and field costs", "Total cost"),
        "community_representation_npv": ("Community representation / liaison", "NPV"),
        "community_representation": ("Community representation / liaison", "Total cost"),
        "blue_carbon_project_planning_npv": ("Blue carbon project planning", "NPV"),
        "blue_carbon_project_planning": ("Blue carbon project planning", "Total cost"),
        "establishing_carbon_rights_npv": ("Establishing carbon rights", "NPV"),
        "establishing_carbon_rights": ("Establishing carbon rights", "Total cost"),
        "validation_npv": ("Validation", "NPV"),
        "validation": ("Validation", "Total cost"),
        "implementation_labor_npv": ("Implementation labor", "NPV"),
        "implementation_labor": ("Implementation labor", "Total cost"),
        "monitoring_npv": ("Monitoring", "NPV"),
        "maintenance_npv": ("Maintenance", "NPV"),
        "monitoring": ("Monitoring", "Total cost"),
        "maintenance": ("Maintenance", "Total cost"),
        "community_benefit_npv": ("Community benefit sharing fund", "NPV"),
        "community_benefit": ("Community benefit sharing fund", "Total cost"),
        "carbon_standard_fees_npv": ("Carbon standard fees", "NPV"),
        "carbon_standard_fees": ("Carbon standard fees", "Total cost"),
        "baseline_reassessment_npv": ("Baseline reassessment", "NPV"),
        "baseline_reassessment": ("Baseline reassessment", "Total cost"),
        "mrv_npv": ("MRV", "NPV"),
        "mrv": ("MRV", "Total cost"),
        "long_term_project_operating_npv": ("Long-term project operating", "NPV"),
        "long_term_project_operating": ("Long-term project operating", "Total cost"),
    }

    # Loop over each cost component and assign
    for col_name, (cost_label, column) in cost_mapping.items():
        df_combinations.loc[index, col_name] = pd.to_numeric(
            cost_estimates.loc[cost_estimates["Cost estimates (USD)"] == cost_label, column]
            .values[0]
            .replace("$", "")
            .replace(",", "")
            .replace("nan", "")
        )

    # Add monitoring_maintenance_npv
    df_combinations.loc[index, "monitoring_maintenance_npv"] = float(
        df_combinations.loc[index, "monitoring_npv"]
    ) + float(df_combinations.loc[index, "maintenance_npv"])
    df_combinations.loc[index, "monitoring_maintenance"] = float(
        df_combinations.loc[index, "monitoring"]
    ) + float(df_combinations.loc[index, "maintenance"])

    # Abatement potential calculations
    sequestration_credits_calculator = SequestrationCreditsCalculator(project)
    # Calculate country abatement potential
    # Safely handle NaN in cost_per_tCO2e
    country_abatement_potential = sequestration_credits_calculator.calculate_abatement_potential()
    df_combinations.loc[index, "country_abatement_potential"] = (
        round(country_abatement_potential) if not math.isnan(country_abatement_potential) else 0
    )
    # Calculate project abatement potential
    net_emissions_reduction_plan = (
        sequestration_credits_calculator.calculate_net_emissions_reductions()
    )
    try:
        project_abatement_potential = round(sum(net_emissions_reduction_plan.values()))
    except Exception as e:
        print(f"Error calculating project abatement potential for {row['project_name']}: {e}")
        project_abatement_potential = 0

    df_combinations.loc[index, "project_abatement_potential"] = project_abatement_potential

    # Weighted cost - divide total_cost and total_cost_npv by country size
    country_size = df_combinations.loc[index, "country_size_ha"]
    df_combinations.loc[index, "total_weighted_cost_npv"] = (
        float(df_combinations.loc[index, "total_cost_npv"]) / country_size
    )
    df_combinations.loc[index, "total_weighted_cost"] = (
        float(df_combinations.loc[index, "total_cost"]) / country_size
    )

    # Safely handle NaN in cost_per_tCO2e
    cost_per_tCO2e = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e) else 0
    )

    # Safely handle NaN in cost_per_tCO2e NPV
    cost_per_tCO2e_npv = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e_npv"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e_npv) else 0
    )
    # Add total revenue, total revenue NPV, and credits issued
    df_combinations.loc[index, "total_revenue"] = pd.to_numeric(
        str(summary["Project summary"].get("Total revenue (non-discounted)", 0))
        .replace("$", "")
        .replace(",", "")
        .replace("nan", "")
    )
    df_combinations.loc[index, "total_revenue_npv"] = pd.to_numeric(
        str(summary["Project summary"].get("Total revenue (NPV)", 0))
        .replace("$", "")
        .replace(",", "")
        .replace("nan", "")
    )
    df_combinations.loc[index, "credits_issued"] = (
        summary["Project summary"].get("Credits issued", "").replace(",", "").replace("nan", "")
    )

    # # Add left_over_after_opex and left_over_after_opex_npv
    df_combinations["leftover_after_opex_npv"] = df_combinations["total_revenue_npv"].astype(
        float
    ) - df_combinations["opex_npv"].astype(float)
    df_combinations["leftover_after_opex"] = df_combinations["total_revenue"].astype(
        float
    ) - df_combinations["opex"].astype(float)

  0%|          | 0/468 [00:00<?, ?it/s]

Error: Credits issued are zero, breakeven cost cannot be calculated.
Error: Breakeven cost for project Australia Salt marsh Conservation  Small could not be calculated 'NoneType' object is not subscriptable
Error: Credits issued are zero, breakeven cost cannot be calculated.
Error: Breakeven cost for project Australia Salt marsh Conservation  Medium could not be calculated 'NoneType' object is not subscriptable
Error: Credits issued are zero, breakeven cost cannot be calculated.
Error: Breakeven cost for project Australia Salt marsh Conservation  Large could not be calculated 'NoneType' object is not subscriptable
Error calculating project abatement potential for Colombia Mangrove Restoration Planting Small: cannot convert float NaN to integer
Error calculating project abatement potential for Colombia Mangrove Restoration Hydrology Small: cannot convert float NaN to integer
Error calculating project abatement potential for Colombia Mangrove Restoration Planting Small: cannot convert fl

In [18]:
def select_columns(df, selector_pair=["Market price", "npv"]):
    """
    Function to select specific columns from a DataFrame
    """
    columns = set(df.columns.values.tolist()) - {
        "country",
        "ecosystem",
        "activity",
        "activity_type",
        "project_size_filter",
        "project_size_ha",
        "price_type",
        "country_size_ha",
        "country_code",
        "base_size",
        "project_name",
    }

    if selector_pair[1] == "npv":
        columns = [col for col in columns if "npv" in col]
        columns.extend(
            [
                "country_total_abatement_potential",
                "country_abatement_potential",
                "project_abatement_potential",
            ]
        )
    else:
        columns = [col for col in columns if "npv" not in col]

    columns.insert(0, "project_name")

    return df[df.price_type == selector_pair[0]][columns]

In [ ]:
def compute_map_tooltip_info(df):
    """compute the average cost per country & the country abatement potential; is this right? we need confirmation from client"""
    df_agg = (
        df.groupby(["country", "ecosystem", "activity", "price_type"])
        .agg(
            {
                "total_cost": "mean",
                "total_cost_npv": "mean",
                "country_abatement_potential": "first",
            }
        )
        .reset_index()
        .groupby(["country", "price_type"])
        .agg(
            {
                "total_cost": "sum",
                "total_cost_npv": "sum",
                "country_abatement_potential": "sum",
            }
        )
        .reset_index()
        .rename(
            columns={
                "total_cost": "country_avg_cost",
                "total_cost_npv": "country_avg_cost_npv",
                "country_abatement_potential": "country_total_abatement_potential",
            },
        )
    )

    return df.join(
        df_agg.set_index(["country", "price_type"]),
        on=["country", "price_type"],
        how="left",
    )

In [21]:
df_combinations.pipe(compute_map_tooltip_info).sort_values(
    by=["country", "ecosystem", "activity", "project_size_filter"]
).pipe(select_columns, ["Market price", "npv"])

,project_name,conservation_planning_npv,carbon_standard_fees_npv,total_weighted_cost_npv,capex_npv,monitoring_maintenance_npv,country_avg_cost_npv,community_representation_npv,feasibility_analysis_npv,opex_npv,...,validation_npv,cost_per_tco2e_npv,blue_carbon_project_planning_npv,total_cost_npv,implementation_labor_npv,maintenance_npv,mrv_npv,country_total_abatement_potential,country_abatement_potential,project_abatement_potential
750,Australia Mangrove Conservation Large,786949,115263,0.007765,1961017,971373.0,53528231.666667,470447,70000,4009181,...,44450,6,102235,5970198,0,0.0,223062,3346658,1618195,1289253
738,Australia Mangrove Conservation Medium,629559,57631,0.005467,1569849,485686.0,53528231.666667,313632,70000,2633371,...,44450,8,102235,4203220,0,0.0,223062,3346658,1618195,644626
726,Australia Mangrove Conservation Small,629559,11526,0.004873,1569849,485686.0,53528231.666667,313632,70000,2176922,...,44450,36,102235,3746770,0,0.0,223062,3346658,1618195,128925
708,Australia Mangrove Restoration Planting Large,629559,7576,0.017012,9197906,2231235.0,53528231.666667,313632,70000,3882163,...,44450,202,102235,13080069,7551095,1745549.0,223062,3346658,503831,80864
714,Australia Mangrove Restoration Hydrology Large,629559,12626,0.02245,12553948,3007035.0,53528231.666667,313632,70000,4707165,...,44450,160,102235,17261114,10907137,2521349.0,223062,3346658,503831,134772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,United States Seagrass Conservation Medium,629559,247,0.004089,1475926,517703.0,163617167.555556,351049,100000,2396943,...,44450,1664,88900,3872869,0,0.0,223062,2400231,104904,2910
186,United States Seagrass Conservation Small,629559,49,0.004087,1475926,517703.0,163617167.555556,351049,100000,2394951,...,44450,8315,88900,3870878,0,0.0,223062,2400231,104904,582
174,United States Seagrass Restoration Planting Large,629559,1640,0.046421,34043555,8028410.0,163617167.555556,351049,100000,9921134,...,44450,3140,88900,43964689,32490666,7510707.0,223062,2400231,104601,17501
162,United States Seagrass Restoration Planting Me...,629559,882,0.026042,18358086,4420269.0,163617167.555556,351049,100000,6305549,...,44450,3342,88900,24663635,16882160,3902566.0,223062,2400231,104601,9224


In [13]:
# export to csv
df_combinations.to_csv(output_path, index=False)

In [15]:
# export as the projects excel tab # this is buggy
with pd.ExcelWriter(
    master_data_path,
    mode="a",
    engine="openpyxl",
    engine_kwargs={"data_only": True},
    if_sheet_exists="replace",
) as writer:
    df_combinations.dropna(subset=["total_cost"]).to_excel(
        writer,
        index=False,
        sheet_name="Projects",
    )

## Scorecard rating:

In [ ]:
# Open csv file
df_combinations = pd.read_csv(output_path)
# fill the nan values with 0
df_combinations.fillna(0, inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code,base_size,...,leftover_after_opex,leftover_after_opex_npv,total_revenue,total_revenu_npv,credits_issued,monitoring_npv,maintenance_npv,monitoring,maintenance,abatement_potential
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,600463.0,290522.0,994000.0,353284.0,166589.0
1,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Open scorecard rating uploaded file.
df_scorecard = pd.read_csv(raw_scorecard_data_path)
# fill the strings ' ' with 0
df_scorecard = df_scorecard.replace(" ", 0)
# fill nans with 0
df_scorecard = df_scorecard.fillna(0)
df_scorecard.head()

In [ ]:
# check the dtypes of the columns
df_scorecard.dtypes
# Set the social_feasibility and the coastal_protection_benefit to float
df_scorecard["social_feasibility"] = df_scorecard["social_feasibility"].astype(float)
df_scorecard["coastal_protection_benefit"] = df_scorecard["coastal_protection_benefit"].astype(
    float
)
df_scorecard.head()

In [ ]:
# Add scoredcard rating to the df_combinations

# Columns to add in df_combinations
columns_to_add = [
    "financial_feasibility",
    "legal_feasibility",
    "implementation_risk_score",
    "social_feasibility",
    "availability_of_experienced_labor",
    "security_rating",
    "availability_of_alternative_funding",
    "coastal_protection_benefit",
    "biodiversity_benefit",
]
df_combinations[columns_to_add] = None

for index, row in df_combinations.iterrows():
    country = row["country"]
    ecosystem = row["ecosystem"]
    # 1. Category: Economic:
    # 1.1 Financial_feasibility (weight 20)
    if (
        round(float(row["total_revenue_npv"]) - float(row["opex_npv"]))
        < float(row["opex_npv"]) * -0.25
    ):
        financial_feasibility = 1
    elif round(float(row["total_revenue_npv"]) - float(row["opex_npv"])) < 0:
        financial_feasibility = 2
    else:
        financial_feasibility = 3
    df_combinations.loc[index, "financial_feasibility"] = financial_feasibility
    # Add financial_feasibility_color
    df_combinations.loc[index, "financial_feasibility_color"] = (
        "Low"
        if financial_feasibility == 1
        else "Medium"
        if financial_feasibility == 2
        else "High"
        if financial_feasibility == 3
        else "Not Available"
    )

    # 2. Non-economic
    # 2.1 Legal feasibility (weight 12)
    # Get the legal feasibility score from the scorecard for the country and ecosystem
    legal_feasibility = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["legal_feasibility"].values[0]
    df_combinations.loc[index, "legal_feasibility"] = legal_feasibility
    # Add legal_feasibility_color
    df_combinations.loc[index, "legal_feasibility_color"] = (
        "High"
        if legal_feasibility == 1
        else "Medium"
        if legal_feasibility == 2
        else "Low"
        if legal_feasibility == 3
        else "Not Available"
    )

    # 2.2 Implementation risk score (weight 12)
    # Get the implementation risk score from the scorecard for the country and ecosystem
    implementation_risk_score = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["implementation_risk_score"].values[0]
    df_combinations.loc[index, "implementation_risk_score"] = implementation_risk_score
    # Add implementation_risk_score_color
    df_combinations.loc[index, "implementation_risk_score_color"] = (
        "High"
        if implementation_risk_score == 1
        else "Medium"
        if implementation_risk_score == 2
        else "Low"
        if implementation_risk_score == 3
        else "Not Available"
    )

    # 2.3 Social feasibility (weight 12)
    # Get the social feasibility score from the scorecard for the country and ecosystem
    social_feasibility = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["social_feasibility"].values[0]
    df_combinations.loc[index, "social_feasibility"] = social_feasibility
    # Add social_feasibility_color
    df_combinations.loc[index, "social_feasibility_color"] = (
        "High"
        if social_feasibility == 1
        else "Medium"
        if social_feasibility == 2
        else "Low"
        if social_feasibility == 3
        else "Not Available"
    )

    # 2.4 Availability of experienced labor (weight 10)
    # Get the availability of experienced labor score from the scorecard
    # for the country and ecosystem
    availability_of_experienced_labor = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["availability_of_experienced_labor"].values[0]
    df_combinations.loc[index, "availability_of_experienced_labor"] = (
        availability_of_experienced_labor
    )
    # Add availability_of_experienced_labor_color
    df_combinations.loc[index, "availability_of_experienced_labor_color"] = (
        "High"
        if availability_of_experienced_labor == 1
        else "Medium"
        if availability_of_experienced_labor == 2
        else "Low"
        if availability_of_experienced_labor == 3
        else "Not Available"
    )

    # 2.5 Security rating (weight 5)
    # Get the security rating score from the scorecard for the country and ecosystem
    security_rating = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["security_rating"].values[0]
    df_combinations.loc[index, "security_rating"] = security_rating
    # Add security_rating_color
    df_combinations.loc[index, "security_rating_color"] = (
        "High"
        if security_rating == 1
        else "Medium"
        if security_rating == 2
        else "Low"
        if security_rating == 3
        else "Not Available"
    )

    # 2.6 Availability of alternative funding (weight 5)
    # Get the availability of alternative funding score from the scorecard
    # for the country and ecosystem
    availability_of_alternative_funding = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["availability_of_alternative_funding"].values[0]
    df_combinations.loc[index, "availability_of_alternative_funding"] = (
        availability_of_alternative_funding
    )
    # Add availability_of_alternative_funding_color
    df_combinations.loc[index, "availability_of_alternative_funding_color"] = (
        "High"
        if availability_of_alternative_funding == 1
        else "Medium"
        if availability_of_alternative_funding == 2
        else "Low"
        if availability_of_alternative_funding == 3
        else "Not Available"
    )

    # 2.7 Coastal protection benefit (weight 3)
    # Get the coastal protection benefit score from the scorecard for the country and ecosystem
    coastal_protection_benefit = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["coastal_protection_benefit"].values[0]
    df_combinations.loc[index, "coastal_protection_benefit"] = coastal_protection_benefit
    # Add coastal_protection_benefit_color
    df_combinations.loc[index, "coastal_protection_benefit_color"] = (
        "High"
        if coastal_protection_benefit == 1
        else "Medium"
        if coastal_protection_benefit == 2
        else "Low"
        if coastal_protection_benefit == 3
        else "Not Available"
    )

    # 2.8 Biodiversity benefit (weight 3)
    # Get the biodiversity benefit score from the scorecard for the country and ecosystem
    biodiversity_benefit = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["biodiversity_benefit"].values[0]
    df_combinations.loc[index, "biodiversity_benefit"] = biodiversity_benefit
    # Add biodiversity_benefit_color
    df_combinations.loc[index, "biodiversity_benefit_color"] = (
        "High"
        if biodiversity_benefit == 1
        else "Medium"
        if biodiversity_benefit == 2
        else "Low"
        if biodiversity_benefit == 3
        else "Not Available"
    )

    # if value is Nan, set to 0 for the calculation
    if math.isnan(legal_feasibility):
        legal_feasibility = 0
    if math.isnan(implementation_risk_score):
        implementation_risk_score = 0
    if math.isnan(social_feasibility):
        social_feasibility = 0
    if math.isnan(availability_of_experienced_labor):
        availability_of_experienced_labor = 0
    if math.isnan(security_rating):
        security_rating = 0
    if math.isnan(availability_of_alternative_funding):
        availability_of_alternative_funding = 0
    if math.isnan(coastal_protection_benefit):
        coastal_protection_benefit = 0
    if math.isnan(biodiversity_benefit):
        biodiversity_benefit = 0

    # Add scorecard_rating
    scorecard_rating = (
        (float(legal_feasibility) * 0.12)
        + (float(implementation_risk_score) * 0.12)
        + (float(social_feasibility) * 0.12)
        + (float(security_rating) * 0.05)
        + (float(availability_of_experienced_labor) * 0.10)
        + (float(availability_of_alternative_funding) * 0.05)
        + (float(coastal_protection_benefit) * 0.03)
        + (float(biodiversity_benefit) * 0.03)
    ) / (0.12 + 0.12 + 0.12 + 0.05 + 0.10 + 0.05 + 0.03 + 0.03)
    df_combinations.loc[index, "scorecard_rating"] = scorecard_rating
    if scorecard_rating > 1 and scorecard_rating <= 1.666:
        df_combinations.loc[index, "scorecard_rating_color"] = "High"
    elif scorecard_rating > 1.666 and scorecard_rating <= 2.333:
        df_combinations.loc[index, "scorecard_rating_color"] = "Medium"
    elif scorecard_rating > 2.333 and scorecard_rating <= 3:
        df_combinations.loc[index, "scorecard_rating_color"] = "Low"
    else:
        df_combinations.loc[index, "scorecard_rating_color"] = "Not Available"

df_combinations.head()

In [ ]:
# export to csv
df_combinations.to_csv(output_scorecard, index=False)